In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk, ImageDraw, ImageFont
import cv2
import threading
import time
import numpy as np
from ultralytics import YOLO

video_path = None
frame_delay = 500  # 2 FPS
model = YOLO("yolo_trening/trening4/weights/best.pt")
class_names = ['oseba_zelo_blizu', 'oseba_blizu', 'oseba_dalec', 
               'vozilo_zelo_blizu', 'vozilo_blizu', 'vozilo_dalec', 
               'ostalo_zelo_blizu', 'ostalo_blizu', 'ostalo_dalec']
danger_label = None
video_label = None
video_frames = []
current_frame_index = 0
playing = False

def select_file():
    global video_path
    video_path = filedialog.askopenfilename(filetypes=[("MP4 files", "*.mp4")])
    print(f"Izbran video: {video_path}")

def get_alpha_for_risk(label):
    if "zelo_blizu" in label:
        return 102  # 40%
    elif "blizu" in label:
        return 51   # 20%
    else:
        return 25   # 10%

def analyze_video():
    global video_frames, current_frame_index
    if not video_path:
        print("Video ni izbran.")
        return

    cap = cv2.VideoCapture(video_path)
    video_frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (1280, 720))
        results = model.predict(source=frame, conf=0.5, iou=0.3, max_det=10, verbose=False)[0]

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(frame_rgb).convert("RGBA")
        overlay = Image.new('RGBA', pil_image.size, (0, 0, 0, 0))
        draw = ImageDraw.Draw(overlay)

        objects = []

        for box in results.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            cls_id = int(box.cls[0].cpu().numpy())
            label = class_names[cls_id]
            objects.append(label)

            alpha = get_alpha_for_risk(label)
            color = (255, 0, 0, alpha)

            draw.rectangle([x1, y1, x2, y2], fill=color)

        combined = Image.alpha_composite(pil_image, overlay).convert("RGB")
        video_frames.append((combined, objects))

    cap.release()
    replay_video()



def replay_video():
    global current_frame_index, playing
    if not video_frames:
        return

    playing = True
    current_frame_index = 0

    def play():
        global current_frame_index, playing
        while playing and current_frame_index < len(video_frames):
            frame, objects = video_frames[current_frame_index]
            frame_tk = ImageTk.PhotoImage(frame)
            video_label.config(image=frame_tk, width=1280, height=720)
            video_label.image = frame_tk

            danger_text = ', '.join(set(objects)) if objects else "Ni nevarnosti"
            danger_label.config(text=f"Danger Level:\n{danger_text}")

            current_frame_index += 1
            time.sleep(frame_delay / 1000.0)

    threading.Thread(target=play).start()


root = tk.Tk()
root.title("ReverseAI")
root.configure(bg="#1c5285")

file_btn = tk.Button(root, text="Izberi video", command=select_file)
file_btn.grid(row=0, column=0)

analyze_btn = tk.Button(root, text="Analiziraj", command=lambda: threading.Thread(target=analyze_video).start())
analyze_btn.grid(row=0, column=1)

danger_label = tk.Label(root, text="Zaznano:\n-", width=25, height=5, bg="white", anchor='nw', justify='left')
danger_label.grid(row=1, column=0)

video_label = tk.Label(root, width=160, height=50)
video_label.grid(row=0, column=2, rowspan=10, columnspan=10)

replay_btn = tk.Button(root, text="Ponovno prikazi", command=replay_video)
replay_btn.grid(row=1, column=1)

root.mainloop()


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\nikkr\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Izbran video: C:/Users/nikkr/Downloads/20250330_130915.mp4
Izbran video: 
Izbran video: 
